<a href="https://colab.research.google.com/github/jobiemay2000/PDPTW_Fair-Model/blob/main/Thesis_PDPTW_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [362]:
 pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [363]:
import folium
from folium.plugins import BeautifyIcon
import pandas as pd
import geopy.distance
import numpy as np

In [364]:
orders = pd.read_csv('https://docs.google.com/spreadsheets/d/1An-AyxvH0go1fONYRO-6G0F-mRnANe2F-cJ4QGwP2bM/export?gid=1301395580&format=csv',index_col=0,)

restos = pd.read_csv('https://docs.google.com/spreadsheets/d/1An-AyxvH0go1fONYRO-6G0F-mRnANe2F-cJ4QGwP2bM/export?gid=0&format=csv',index_col=0,)

riders = pd.read_csv('https://docs.google.com/spreadsheets/d/1An-AyxvH0go1fONYRO-6G0F-mRnANe2F-cJ4QGwP2bM/export?gid=1625475323&format=csv', index_col=0,)

In [365]:
P = [ i for i in orders.index] #list of pickup orders
D = [ i+100 for i in P ] #list of dropoff customers 
R = [r for r in riders.index]
O = [(o,d) for o,d in zip(P,D)] #Os order,dropoff (1,101),(2,102)


P_loc = [[
          restos.loc[orders.loc[i]['restaurant']]['latitude'],
          restos.loc[orders.loc[i]['restaurant']]['longitude']
         ] for i in P] #resto location
D_loc = [[x,y] for x,y in zip(orders['latitude'],orders['longitude'] )] #customer location

N = [0] + P + D 
N_loc = [[7.08, 125.61]] + P_loc + D_loc
loc_df = pd.DataFrame(N_loc, columns=['lat', 'long'], index=N)

q = {i:orders.loc[i].demand for i in orders.index}

for n in D:
    q[n] = (-1)*q[n-100]
q[0] = 0


In [366]:
colors = ["red", "blue", "green", "purple", "orange", "darkred",
          "pink", "lightblue", "lightgreen", "gray",'#F6CEF5', "black","darkgreen","cadetblue",'#61210B',"red", "blue","lightgray","green"]


In [367]:
bigM = 9999
def distance(r,i,j):   
  if i == 0:
    a = geopy.distance.geodesic((riders.loc[r].latitude,riders.loc[r].longitude),(loc_df.loc[j].lat,loc_df.loc[j].long)).km * 1000 
  else:
    a = geopy.distance.geodesic((loc_df.loc[i].lat,loc_df.loc[i].long),(loc_df.loc[j].lat,loc_df.loc[j].long)).km * 1000
  return a

In [368]:
map_i1 = folium.Map(location=[7.076932516986668, 125.61325169717809], min_zoom=14,max_zoom = 16,tiles='cartodbpositron')





bundle = {
  'rd4': [0, 5, 105, 0],'rd5': [0, 4, 10, 104, 12, 110, 112, 0], # 132.597, 0.2148
    
  'rd26': [0, 15, 18, 115, 118, 0], 'rd7': [0, 9, 14, 114, 109, 0], 'rd1': [0, 1, 101, 0], #195.447, 0.2873

  'rd2': [0, 20, 6, 106, 120, 0], 'rd28': [0, 2, 7, 102, 107, 0], 'rd30': [0, 17, 8, 108, 117, 0], #202.704, 4.7201

  'rd22': [0, 13, 3, 103, 113, 0], 'rd12': [0, 11, 111, 0], 'rd10': [0, 19, 119, 16, 116, 0] #197.451, 0.4084


}
all_riders_grp = folium.FeatureGroup('Riders', show=False)
map_i1.add_child(all_riders_grp)
for key in riders.index:
  a = folium.Marker(location=[riders.loc[key]['latitude'], riders.loc[key]['longitude']],
                popup=key, opacity=0.8, icon=folium.Icon(color='green',icon='motorcycle', prefix='fa'))
  a.add_to(all_riders_grp)

riders_notchosen = folium.FeatureGroup('Riders Coded', show=False)
map_i1.add_child(riders_notchosen)
idle_riders = {i for i in riders.index if i not in bundle}
for key in idle_riders:
  if riders.loc[key].idle < 5:
    a = folium.Marker(location=[riders.loc[key]['latitude'], riders.loc[key]['longitude']],
                popup=key, opacity=0.7, icon=folium.Icon(color='gray',icon='motorcycle', prefix='fa'))
    a.add_to(riders_notchosen)
  else:
    a = folium.Marker(location=[riders.loc[key]['latitude'], riders.loc[key]['longitude']],
                popup=key, opacity=0.7, icon=folium.Icon(color='green',icon='motorcycle', prefix='fa'))
    a.add_to(riders_notchosen)
blue_red1 = folium.FeatureGroup('All Nodes', show=False)
map_i1.add_child(blue_red1) 
for ix, o in orders.iterrows():
  lati = o['latitude']
  longi = o['longitude']
  folium.Marker(location=[lati,longi], 
                  popup=str(ix), 
                 opacity=0.9,  icon=folium.Icon(color='blue', icon='home', prefix='fa')).add_to(blue_red1)
for ix, o in restos.iterrows():
  lati = o['latitude']
  longi = o['longitude']
  folium.Marker(location=[lati,longi], 
                  popup=str(ix), 
                 opacity=0.9,  icon=folium.Icon(color='red', icon='cutlery', prefix='fa')).add_to(blue_red1)


cl = {}
for key in bundle:
  cl[key] = folium.FeatureGroup(str(key))
  map_i1.add_child(cl[key])

cix = 0
for (key,route) in bundle.items():
  if riders.loc[key].idle >= 30:
    a = folium.Marker(location=[riders.loc[key]['latitude'], riders.loc[key]['longitude']],
                popup=key, opacity=0.9, icon=folium.Icon(color='orange',icon='motorcycle', prefix='fa'))
    a.add_to(cl[key]) 
  else:
    a = folium.Marker(location=[riders.loc[key]['latitude'], riders.loc[key]['longitude']],
                popup=key, opacity=0.9, icon=folium.Icon(color='green',icon='motorcycle', prefix='fa'))
    a.add_to(cl[key]) 

  for node in route:
    lati = loc_df.loc[node].lat
    longi= loc_df.loc[node].long
    if node > 100:
      a = folium.Marker(location=[lati,longi], 
                  popup=str(node) + ' from ' + str(orders.loc[node-100].restaurant) + '. Load:{}'.format(orders.loc[node-100].demand), 
                 opacity=0.9,  icon=folium.Icon(color='blue', icon='home', prefix='fa'))
      a.add_to(cl[key]) 
    else:
      if node != 0:
        a=folium.Marker(location=[lati,longi],popup=orders.loc[node].restaurant, 
                      opacity=0.9, icon=folium.Icon(color='red', icon='cutlery', prefix='fa'),
                      tooltip='click')
        a.add_to(cl[key]) 

  #routes
  for i,j in zip(route[:-1],route[1:-1]):
    if i ==0: #rider
      orig = (riders.loc[key]['latitude'] ,riders.loc[key]['longitude'])
    else:
      orig = (loc_df.loc[i].lat,loc_df.loc[i].long )

    destin = (loc_df.loc[j].lat,loc_df.loc[j].long)
    a = folium.PolyLine([orig,destin],
                  color=colors[cix],
                  weight=5,
                  opacity=0.9)
    a.add_to(cl[key]) 
 
  print(route,colors[cix])
  cix+=1


## BASE MODEL i1
bundle_basemodel_1 = {
  'rd6': [0, 4, 10, 104, 12, 110, 112, 0], 'rd3': [0, 5, 105, 0], #130.682, 0.7067
  
  'rd20': [0, 15, 115, 1, 101, 0], 'rd8': [0, 18, 14, 9, 118, 114, 109, 0], #156.610, 12.8463

  'rd29': [0, 2, 7, 17, 102, 107, 117, 0], 'rd28': [0, 6, 20, 106, 8, 108, 120, 0], # 152.474, 189.5696

  'rd9': [0, 19, 119, 16, 116, 0], 'rd27': [0, 3, 103, 11, 13, 111, 113, 0] #143.025, 17.2191
  
}

base_mod_i1 = folium.FeatureGroup('PDPTW_mod_i1', show=False)
map_i1.add_child(base_mod_i1)

idle_riders = {i for i in riders.index if i not in bundle_basemodel_1}
for key in idle_riders:
  if riders.loc[key].idle >30 :
    a = folium.Marker(location=[riders.loc[key]['latitude'], riders.loc[key]['longitude']],
                popup=key, opacity=0.7, icon=folium.Icon(color='orange',icon='motorcycle', prefix='fa'))
    a.add_to(base_mod_i1)
  else:
    a = folium.Marker(location=[riders.loc[key]['latitude'], riders.loc[key]['longitude']],
                popup=key, opacity=0.7, icon=folium.Icon(color='green',icon='motorcycle', prefix='fa'))
    a.add_to(base_mod_i1)


cix = 0
for (key,route) in bundle_basemodel_1.items():
  if riders.loc[key].idle < 5:
    a = folium.Marker(location=[riders.loc[key]['latitude'], riders.loc[key]['longitude']],
                popup=key, opacity=0.9, icon=folium.Icon(color='gray',icon='motorcycle', prefix='fa'))
    a.add_to(base_mod_i1) 
  else:
    a = folium.Marker(location=[riders.loc[key]['latitude'], riders.loc[key]['longitude']],
                popup=key, opacity=0.9, icon=folium.Icon(color='green',icon='motorcycle', prefix='fa'))
    a.add_to(base_mod_i1) 

  for node in route:
    lati = loc_df.loc[node].lat
    longi= loc_df.loc[node].long
    if node > 100:
      a = folium.Marker(location=[lati,longi], 
                  popup=str(node) + ' from ' + str(orders.loc[node-100].restaurant), 
                 opacity=0.9,  icon=folium.Icon(color='blue', icon='home', prefix='fa'))
      a.add_to(base_mod_i1) 
    else:
      if node != 0:
        a=folium.Marker(location=[lati,longi],popup=orders.loc[node].restaurant, 
                      opacity=0.9, icon=folium.Icon(color='red', icon='cutlery', prefix='fa'),
                      tooltip='click')
        a.add_to(base_mod_i1) 

  #routes
  for i,j in zip(route[:-1],route[1:-1]):
    if i ==0: #rider
      orig = (riders.loc[key]['latitude'] ,riders.loc[key]['longitude'])
    else:
      orig = (loc_df.loc[i].lat,loc_df.loc[i].long )

    destin = (loc_df.loc[j].lat,loc_df.loc[j].long)
    a = folium.PolyLine([orig,destin],
                  color=colors[cix],
                  weight=5,
                  opacity=0.9)
    a.add_to(base_mod_i1) 
 
  cix+=1

map_i1.add_child(folium.LayerControl())
map_i1

[0, 5, 105, 0] red
[0, 4, 10, 104, 12, 110, 112, 0] blue
[0, 15, 18, 115, 118, 0] green
[0, 9, 14, 114, 109, 0] purple
[0, 1, 101, 0] orange
[0, 20, 6, 106, 120, 0] darkred
[0, 2, 7, 102, 107, 0] pink
[0, 17, 8, 108, 117, 0] lightblue
[0, 13, 3, 103, 113, 0] lightgreen
[0, 11, 111, 0] gray
[0, 19, 119, 16, 116, 0] #F6CEF5


In [369]:
load = dict( (r,[0]) for r in bundle)
route_cost=dict( (r,0) for r in bundle)
route_dist=dict( (r,0) for r in bundle)

fcost_rider = 45
fcost_travel = 7/1000
    
for (r,route) in bundle.items():
  route_cost[r] = round(fcost_rider + (fcost_travel * sum(distance(r,i,j) for i,j in zip(route[:-1],route[1:-1]))) ,2)
  route_dist[r] = round(sum(distance(r,i,j) for i,j in zip(route[:-1],route[1:-1])),2)
  for ix,node in enumerate(route[1:-1]):
    load[r] += [load[r][ix] + q[node]]

In [370]:
clust=dict( (r,'' ) for r in bundle)
runtime=dict( (r,'') for r in bundle)
rider=dict( (r,r) for r in bundle)

df = pd.DataFrame([clust, rider, bundle,load, route_cost, route_dist,runtime]).T
df.columns = ['Cluster','Rider','Route', 'Load', 'Cost', 'Distance','Run Time']

df.set_index("Cluster", inplace = True)
df.at['Total', 'Distance'] = df['Distance'].sum()
df.at['Total', 'Cost'] = df['Cost'].sum()

In [371]:
df

,Rider,Route,Load,Cost,Distance,Run Time
Cluster,,,,,,
,rd4,"[0, 5, 105, 0]","[0, 1, 0]",62.34,2476.61,
,rd5,"[0, 4, 10, 104, 12, 110, 112, 0]","[0, 4, 5, 1, 3, 2, 0]",70.26,3608.7,
,rd26,"[0, 15, 18, 115, 118, 0]","[0, 2, 5, 3, 0]",60.66,2237.0,
,rd7,"[0, 9, 14, 114, 109, 0]","[0, 2, 7, 2, 0]",66.09,3012.76,
,rd1,"[0, 1, 101, 0]","[0, 10, 0]",68.7,3385.59,
,rd2,"[0, 20, 6, 106, 120, 0]","[0, 3, 5, 3, 0]",71.96,3851.0,
,rd28,"[0, 2, 7, 102, 107, 0]","[0, 3, 6, 3, 0]",64.23,2746.68,
,rd30,"[0, 17, 8, 108, 117, 0]","[0, 4, 8, 4, 0]",66.52,3074.28,
,rd22,"[0, 13, 3, 103, 113, 0]","[0, 1, 5, 1, 0]",72.05,3863.78,


In [372]:
load_base = dict( (r,[0]) for r in bundle_basemodel_1)
route_cost=dict( (r,0) for r in bundle_basemodel_1)
route_dist=dict( (r,0) for r in bundle_basemodel_1)

fcost_rider = 45
fcost_travel = 7/1000
    
for (r,route) in bundle_basemodel_1.items():
  route_cost[r] = round(fcost_rider + (fcost_travel * sum(distance(r,i,j) for i,j in zip(route[:-1],route[1:-1]))) ,2)
  route_dist[r] = round(sum(distance(r,i,j) for i,j in zip(route[:-1],route[1:-1])),2)
  for ix,node in enumerate(route[1:-1]):
    load_base[r] += [load_base[r][ix] + q[node]]


clust=dict( (r,'' ) for r in bundle_basemodel_1)
runtime=dict( (r,'') for r in bundle_basemodel_1)
rider = dict( (r,r) for r in bundle_basemodel_1)

df_base = pd.DataFrame([clust, rider,bundle_basemodel_1,load_base, route_cost, route_dist,runtime]).T
df_base.columns = ['Cluster','Rider','Route', 'Load', 'Cost', 'Distance','Run Time']

df_base.set_index('Cluster', inplace = True)
df_base.at['Total', 'Distance'] = df_base['Distance'].sum()
df_base.at['Total', 'Cost'] = df_base['Cost'].sum()

In [373]:
df_base

,Rider,Route,Load,Cost,Distance,Run Time
Cluster,,,,,,
,rd6,"[0, 4, 10, 104, 12, 110, 112, 0]","[0, 4, 5, 1, 3, 2, 0]",69.46,3493.83,
,rd3,"[0, 5, 105, 0]","[0, 1, 0]",61.22,2317.84,
,rd20,"[0, 15, 115, 1, 101, 0]","[0, 2, 0, 10, 0]",79.01,4859.27,
,rd8,"[0, 18, 14, 9, 118, 114, 109, 0]","[0, 3, 8, 10, 7, 2, 0]",77.6,4656.45,
,rd29,"[0, 2, 7, 17, 102, 107, 117, 0]","[0, 3, 6, 10, 7, 4, 0]",73.86,4122.68,
,rd28,"[0, 6, 20, 106, 8, 108, 120, 0]","[0, 2, 5, 3, 7, 3, 0]",78.62,4802.22,
,rd9,"[0, 19, 119, 16, 116, 0]","[0, 4, 0, 2, 0]",61.62,2374.62,
,rd27,"[0, 3, 103, 11, 13, 111, 113, 0]","[0, 4, 0, 3, 4, 1, 0]",81.4,5200.35,
Total,NaN,NaN,NaN,582.79,31827.26,NaN


# **Instance 2**

In [374]:
orders2 = pd.read_csv('https://docs.google.com/spreadsheets/d/1An-AyxvH0go1fONYRO-6G0F-mRnANe2F-cJ4QGwP2bM/export?gid=1470933557&format=csv', index_col=0)

restos2 = pd.read_csv('https://docs.google.com/spreadsheets/d/1An-AyxvH0go1fONYRO-6G0F-mRnANe2F-cJ4QGwP2bM/export?gid=1241448397&format=csv',index_col=0)

riders2 = pd.read_csv('https://docs.google.com/spreadsheets/d/1An-AyxvH0go1fONYRO-6G0F-mRnANe2F-cJ4QGwP2bM/export?gid=2069000863&format=csv',index_col=0)

In [375]:
P = [ i for i in orders2.index] #list of pickup orders
D = [ i+100 for i in P ] #list of dropoff customers 
R = [r for r in riders2.index]
O = [(o,d) for o,d in zip(P,D)] #Os order,dropoff (1,101),(2,102)

P_loc = [[
          restos2.loc[orders2.loc[i]['restaurant']]['latitude'],
          restos2.loc[orders2.loc[i]['restaurant']]['longitude']
         ] for i in P] #resto location
D_loc = [[x,y] for x,y in zip(orders2['latitude'],orders2['longitude'] )] #customer location

N = [0] + P + D 
N_loc = [[7.08, 125.61]] + P_loc + D_loc
loc_df2 = pd.DataFrame(N_loc, columns=['lat', 'long'], index=N)

q2 = {i:orders2.loc[i].demand for i in orders2.index}

for n in D:
    q2[n] = (-1)*q2[n-100]
q2[0] = 0

In [376]:
def distance2(r,i,j):
  if i == 0:
    a = geopy.distance.geodesic((riders2.loc[r].latitude,riders2.loc[r].longitude),(loc_df2.loc[j].lat,loc_df2.loc[j].long)).km * 1000 
  else:
    a = geopy.distance.geodesic((loc_df2.loc[i].lat,loc_df2.loc[i].long),(loc_df2.loc[j].lat,loc_df2.loc[j].long)).km * 1000
  return a

In [377]:
#create map
map_i2 = folium.Map(location=[7.076932516986668, 125.61325169717809], min_zoom=13, max_zoom = 16, tiles='cartodbpositron')

bundle2 = {
    'rd23': [0, 22, 11, 111, 122, 0], 'rd1': [0, 1, 101, 12, 112, 0], 'rd5': [0, 2, 102, 0], 'rd30': [0, 19, 3, 103, 119, 0], #1 275.032, 3.3321
    
    'rd25': [0, 25, 24, 125, 124, 0], 'rd26': [0, 26, 23, 123, 126, 0], #2 145.680 0.1442
    
    'rd21': [0, 4, 104, 0], 'rd9': [0, 13, 20, 113, 120, 0], 'rd22': [0, 15, 115, 0], 'rd10': [0, 14, 16, 18, 114, 118, 116, 0], #3 286.462, 8.7434

    
    'rd19': [0, 27, 10, 17, 117, 110, 127, 0], 'rd32': [0, 21, 121, 0], 'rd18': [0, 8, 9, 109, 108, 0], #4 204.271 1.6194

    'rd16': [0, 7, 107, 6, 106, 0], 
    'rd13': [0, 5, 105, 0], #5 126.822 0.1478
}

all_riders_grp2 = folium.FeatureGroup('Riders', show=False)
map_i2.add_child(all_riders_grp2)
for key in riders2.index:
  a = folium.Marker(location=[riders2.loc[key]['latitude'], riders2.loc[key]['longitude']],
                popup=key, opacity=0.8, icon=folium.Icon(color='green',icon='motorcycle', prefix='fa'))
  a.add_to(all_riders_grp2)

riders_notchosen2 = folium.FeatureGroup('Riders Coded', show=False)
map_i2.add_child(riders_notchosen2)

idle_riders = {i for i in riders2.index if i not in bundle2}
for key in idle_riders:
  if riders2.loc[key].idle < 5:
    folium.Marker(location=[riders2.loc[key]['latitude'], riders2.loc[key]['longitude']],
                popup=key, opacity=0.7, icon=folium.Icon(color='gray',icon='motorcycle', prefix='fa')).add_to(riders_notchosen2)
  else:
    folium.Marker(location=[riders2.loc[key]['latitude'], riders2.loc[key]['longitude']],
                popup=key, opacity=0.7, icon=folium.Icon(color='green',icon='motorcycle', prefix='fa')).add_to(riders_notchosen2)

blue_red2 = folium.FeatureGroup('All Nodes', show=False)
map_i2.add_child(blue_red2) 
for ix, o in orders2.iterrows():
  lati = o['latitude']
  longi = o['longitude']
  folium.Marker(location=[lati,longi], 
                  popup=str(ix), 
                 opacity=0.9,  icon=folium.Icon(color='blue', icon='home', prefix='fa')).add_to(blue_red2)
for ix, o in restos2.iterrows():
  lati = o['latitude']
  longi = o['longitude']
  folium.Marker(location=[lati,longi], 
                  popup=str(ix), 
                 opacity=0.9,  icon=folium.Icon(color='red', icon='cutlery', prefix='fa')).add_to(blue_red2)



cl2 = {}
for key in bundle2:
  cl2[key] = folium.FeatureGroup(str(key))
  map_i2.add_child(cl2[key])

cix = 0
for (key,route) in bundle2.items():
  if riders2.loc[key].idle >= 30:
    folium.Marker(location=[riders2.loc[key]['latitude'], riders2.loc[key]['longitude']],
                popup=key, opacity=0.9, icon=folium.Icon(color='orange',icon='motorcycle', prefix='fa')).add_to(cl2[key])
  else:
    folium.Marker(location=[riders2.loc[key]['latitude'], riders2.loc[key]['longitude']],
                popup=key, opacity=0.9, icon=folium.Icon(color='green',icon='motorcycle', prefix='fa')).add_to(cl2[key])
                
  for node in route:
    lati = loc_df2.loc[node].lat
    longi= loc_df2.loc[node].long
    if node > 100:
      a = folium.Marker(location=[lati,longi], 
                  popup=str(node) + ' from ' + str(orders2.loc[node-100].restaurant) + '. Load:{}'.format(orders2.loc[node-100].demand), 
                 opacity=0.9,  icon=folium.Icon(color='blue', icon='home', prefix='fa'))
      a.add_to(cl2[key])
      
    else:
      if node != 0:
        a = folium.Marker(location=[lati,longi],popup=orders2.loc[node].restaurant, 
                      opacity=0.9, icon=folium.Icon(color='red', icon='cutlery', prefix='fa'),
                      tooltip='click')

        a.add_to(cl2[key])
        

  
  for i,j in zip(route[:-1],route[1:-1]):
    if i ==0: #rider
      orig = (riders2.loc[key]['latitude'] ,riders2.loc[key]['longitude'])
    else:
      orig = (loc_df2.loc[i].lat,loc_df2.loc[i].long )

    destin = (loc_df2.loc[j].lat,loc_df2.loc[j].long)
    folium.PolyLine([orig,destin],
                  color=colors[cix],
                  weight=5,
                  opacity=0.9).add_to(cl2[key])
  
  # print(route,colors[cix])                
  cix+=1

## BASE i2
bundle_basemodel_2 = { #same lang ang plot

'rd23': [0, 1, 101, 12, 112, 0], 'rd5': [0, 2, 102, 22, 11, 111, 122, 0], 'rd6': [0, 19, 3, 103, 119, 0], # 239.940, 98.6459

'rd24': [0, 26, 23, 123, 126, 0], #2 143.546, 0.3698
'rd26': [0, 24, 25, 125, 124, 0] ,

'rd11': [0, 20, 15, 120, 115, 0], #3 232.837 164.5794
'rd10': [0, 4, 14, 104, 114, 0],
'rd9': [0, 13, 16, 18, 113, 118, 116, 0], 


'rd32': [0, 21, 9, 109, 121, 0], # 4 167.151, 6.3794
'rd17': [0, 8, 108, 10, 17, 27, 117, 110, 127, 0],

'rd14': [0, 5, 105, 6, 106, 7, 107, 0], #84.376, 0.1257
}
base_mod_i2 = folium.FeatureGroup('PDPTW_mod_i2',show=False)
map_i2.add_child(base_mod_i2)

idle_riders_base2 = {i for i in riders.index if i not in bundle_basemodel_2}
for key in idle_riders_base2:
  if riders.loc[key].idle >30 :
    a = folium.Marker(location=[riders2.loc[key]['latitude'], riders2.loc[key]['longitude']],
                popup=key, opacity=0.7, icon=folium.Icon(color='orange',icon='motorcycle', prefix='fa'))
    a.add_to(base_mod_i2)
  else:
    a = folium.Marker(location=[riders2.loc[key]['latitude'], riders2.loc[key]['longitude']],
                popup=key, opacity=0.7, icon=folium.Icon(color='green',icon='motorcycle', prefix='fa'))
    a.add_to(base_mod_i2)


cix = 0
for (key,route) in bundle_basemodel_2.items():
  if riders2.loc[key].idle < 5:
    a = folium.Marker(location=[riders2.loc[key]['latitude'], riders2.loc[key]['longitude']],
                popup=key, opacity=0.9, icon=folium.Icon(color='gray',icon='motorcycle', prefix='fa'))
    a.add_to(base_mod_i2) 
  else:
    a = folium.Marker(location=[riders2.loc[key]['latitude'], riders2.loc[key]['longitude']],
                popup=key, opacity=0.9, icon=folium.Icon(color='green',icon='motorcycle', prefix='fa'))
    a.add_to(base_mod_i2) 

  for node in route:
    lati = loc_df2.loc[node].lat
    longi= loc_df2.loc[node].long
    if node > 100:
      a = folium.Marker(location=[lati,longi], 
                  popup=str(node) + ' from ' + str(orders2.loc[node-100].restaurant), 
                 opacity=0.9,  icon=folium.Icon(color='blue', icon='home', prefix='fa'))
      a.add_to(base_mod_i2) 
    else:
      if node != 0:
        a=folium.Marker(location=[lati,longi],popup=orders2.loc[node].restaurant, 
                      opacity=0.9, icon=folium.Icon(color='red', icon='cutlery', prefix='fa'),
                      tooltip='click')
        a.add_to(base_mod_i2) 

  #routes
  for i,j in zip(route[:-1],route[1:-1]):
    if i ==0: #rider
      orig = (riders2.loc[key]['latitude'] ,riders2.loc[key]['longitude'])
    else:
      orig = (loc_df2.loc[i].lat,loc_df2.loc[i].long )

    destin = (loc_df2.loc[j].lat,loc_df2.loc[j].long)
    a = folium.PolyLine([orig,destin],
                  color=colors[cix],
                  weight=5,
                  opacity=0.9)
    a.add_to(base_mod_i2) 
 
  cix+=1

map_i2.add_child(folium.LayerControl())
map_i2

In [378]:
load2 = dict( (r,[0]) for r in bundle2)
route_cost2=dict( (r,0) for r in bundle2)
route_dist2=dict( (r,0) for r in bundle2)

fcost_rider = 45
fcost_travel = 7/1000
    
for (r,route) in bundle2.items():
  route_cost2[r] = round(fcost_rider + (fcost_travel * sum(distance2(r,i,j) for i,j in zip(route[:-1],route[1:-1]))),2)
  route_dist2[r] = round(sum(distance2(r,i,j) for i,j in zip(route[:-1],route[1:-1])),2)
  for ix,node in enumerate(route[1:-1]):
    load2[r] += [load2[r][ix] + q2[node]]

In [379]:
clust=dict( (r,'' ) for r in bundle2)
runtime=dict( (r,'') for r in bundle2)
rider=dict( (r,r) for r in bundle2)

df2 = pd.DataFrame([clust, rider,bundle2,load2, route_cost2, route_dist2, runtime]).T
df2.columns = ['Cluster','Rider','Route', 'Load', 'Cost', 'Distance', 'Run Time']
df2.set_index("Cluster", inplace = True)
df2.at['Total', 'Distance'] = df2['Distance'].sum()
df2.at['Total', 'Cost'] = df2['Cost'].sum()

In [380]:
df2

,Rider,Route,Load,Cost,Distance,Run Time
Cluster,,,,,,
,rd23,"[0, 22, 11, 111, 122, 0]","[0, 4, 9, 4, 0]",71.66,3809.11,
,rd1,"[0, 1, 101, 12, 112, 0]","[0, 10, 0, 4, 0]",64.18,2739.97,
,rd5,"[0, 2, 102, 0]","[0, 4, 0]",59.8,2114.7,
,rd30,"[0, 19, 3, 103, 119, 0]","[0, 2, 4, 2, 0]",79.39,4912.26,
,rd25,"[0, 25, 24, 125, 124, 0]","[0, 1, 4, 3, 0]",73.05,4006.65,
,rd26,"[0, 26, 23, 123, 126, 0]","[0, 2, 4, 2, 0]",72.63,3947.62,
,rd21,"[0, 4, 104, 0]","[0, 4, 0]",58.61,1944.24,
,rd9,"[0, 13, 20, 113, 120, 0]","[0, 4, 7, 3, 0]",73.8,4114.38,
,rd22,"[0, 15, 115, 0]","[0, 3, 0]",77.75,4678.81,


In [381]:
load2_base = dict( (r,[0]) for r in bundle_basemodel_2)
route_cost2=dict( (r,0) for r in bundle_basemodel_2)
route_dist2=dict( (r,0) for r in bundle_basemodel_2)

fcost_rider = 45
fcost_travel = 7/1000
    
for (r,route) in bundle_basemodel_2.items():
  route_cost2[r] = round(fcost_rider + (fcost_travel * sum(distance2(r,i,j) for i,j in zip(route[:-1],route[1:-1]))),2)
  route_dist2[r] = round(sum(distance2(r,i,j) for i,j in zip(route[:-1],route[1:-1])),2)
  for ix,node in enumerate(route[1:-1]):
    load2_base[r] += [load2_base[r][ix] + q2[node]]


clust=dict( (r,'' ) for r in bundle_basemodel_2)
runtime=dict( (r,'') for r in bundle_basemodel_2)
rider=dict( (r,r) for r in bundle_basemodel_2)

df2_base = pd.DataFrame([clust, rider,bundle_basemodel_2,load2_base, route_cost2, route_dist2,runtime]).T
df2_base.columns = ['Cluster','Rider','Route', 'Load', 'Cost', 'Distance', 'Run Time']
df2_base.set_index("Cluster", inplace = True)
df2_base.at['Total', 'Distance'] = df2_base['Distance'].sum()
df2_base.at['Total', 'Cost'] = df2_base['Cost'].sum()
df2_base

,Rider,Route,Load,Cost,Distance,Run Time
Cluster,,,,,,
,rd23,"[0, 1, 101, 12, 112, 0]","[0, 10, 0, 4, 0]",62.93,2561.08,
,rd5,"[0, 2, 102, 22, 11, 111, 122, 0]","[0, 4, 0, 4, 9, 4, 0]",99.9,7843.22,
,rd6,"[0, 19, 3, 103, 119, 0]","[0, 2, 4, 2, 0]",77.11,4587.2,
,rd24,"[0, 26, 23, 123, 126, 0]","[0, 2, 4, 2, 0]",70.91,3701.52,
,rd26,"[0, 24, 25, 125, 124, 0]","[0, 3, 4, 3, 0]",72.63,3947.85,
,rd11,"[0, 20, 15, 120, 115, 0]","[0, 3, 6, 3, 0]",100.78,7968.97,
,rd10,"[0, 4, 14, 104, 114, 0]","[0, 4, 6, 2, 0]",59.07,2010.58,
,rd9,"[0, 13, 16, 18, 113, 118, 116, 0]","[0, 4, 5, 8, 4, 1, 0]",72.98,3997.14,
,rd32,"[0, 21, 9, 109, 121, 0]","[0, 5, 8, 5, 0]",76.55,4507.72,


In [382]:
df.to_latex('df_i1.tex')
df2.to_latex('df_i2.tex')

df_base.to_latex('df_i1_base.tex')
df2_base.to_latex('df_i2_base.tex')

In [383]:
riders.to_latex('i_1_riders.tex')
orders.to_latex('i_1_orders.tex')
restos.to_latex('i_1_restos.tex')
riders2.to_latex('i_2_riders.tex')
orders2.to_latex('i_2_orders.tex')
restos2.to_latex('i_2_restos.tex')

In [384]:
map_i1.save('map1.html')
map_i2.save('map2.html')